# NLP: Sentiment analysis (v2)

ML Sample of Natural Language Processing.

- For environment test and confirmation.

## Dataset

Bag of Words Meets Bags of Popcorn
> Use Google's Word2Vec for movie reviews

https://www.kaggle.com/c/word2vec-nlp-tutorial

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import contractions 
import re
from nltk.corpus import stopwords
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score

from sklearn.svm import SVC


In [2]:
pd.set_option("display.max_colwidth", 200)

nltk.download('stopwords')

STOPWORDS = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Load Train Dataset
df_train = pd.read_csv(
    "./raw_data/labeledTrainData.tsv",
    delimiter="\t",
    na_filter=False
)

display(df_train.head(10))

,id,sentiment,review
0,5814_8,1,"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just wa..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds i..."
2,7759_3,0,"The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like ¨Jurassik Park¨, an..."
3,3630_4,0,"It must be assumed that those who praised this film (\the greatest filmed opera ever,\"" didn't I read somewhere?) either don't care for opera, don't care for Wagner, or don't care about anything e..."
4,9495_8,1,"Superbly trashy and wondrously unpretentious 80's exploitation, hooray! The pre-credits opening sequences somewhat give the false impression that we're dealing with a serious and harrowing drama, ..."
5,8196_8,1,"I dont know why people think this is such a bad movie. Its got a pretty good plot, some good action, and the change of location for Harry does not hurt either. Sure some of its offensive and gratu..."
6,7166_2,0,"This movie could have been very good, but comes up way short. Cheesy special effects and so-so acting. I could have looked past that if the story wasn't so lousy. If there was more of a background..."
7,10633_1,0,I watched this video at a friend's house. I'm glad I did not waste money buying this one. The video cover has a scene from the 1975 movie Capricorn One. The movie starts out with several clips of ...
8,319_1,0,"A friend of mine bought this film for £1, and even then it was grossly overpriced. Despite featuring big names such as Adam Sandler, Billy Bob Thornton and the incredibly talented Burt Young, this..."
9,8713_10,1,"<br /><br />This movie is full of references. Like \Mad Max II\"", \""The wild one\"" and many others. The ladybug´s face it´s a clear reference (or tribute) to Peter Lorre. This movie is a masterpie..."


In [4]:
# Methods preparation
def clean_text(text: str) -> str:
    """Clean text."""
    text = _clean_text_html_tag_removing(text)
    text = _clean_text_expand_contractions(text)
    text = _clean_text_lowercase_conversion(text)
    text = _clean_text_non_alpha_numeric_replacing(text)
    text = _clean_text_stopwords_removing(text)
    return text
    

def _clean_text_html_tag_removing(text: str) -> str:
    """Clean text with removing HTML tags."""
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


def _clean_text_expand_contractions(text: str) -> str:
    """Clean text with expansion contractions."""
    return contractions.fix(text)


def _clean_text_lowercase_conversion(text: str) -> str:
    """Clean text with lower case conversion."""
    return text.lower()


def _clean_text_non_alpha_numeric_replacing(text: str) -> str:
    """Clean text by replacing non-alphanumeric characters with spaces."""
    return re.sub("[^a-zA-Z0-9]", " ", text)


def _clean_text_stopwords_removing(text: str) -> str:
    """Clean text with removing stopwords."""
    words = text.split()
    words = [
        word for word in words if word not in STOPWORDS
    ]
    return ' '.join(words)


def evaluate_trained_model(
    model: BaseEstimator,
    X_val_data: list,
    y_val_data: list
) -> None:
    """Evaluate a trained Machine Learning model using various metrics

    This function provides:
    - Accuracy Score: Measures how accurately the class labels are predicted.
    - Precision Score: Evaluates how many of the items predicted as positive are actually positive.
    - Confusion Matrix: Provides a matrix representing TP, FP, FN, TN for each class.
    - Classification Report: Generates a detailed report including Precision, Recall, F1-score, and Support for each class.

    Args:
        model: Trained machine learning model.
        X_test_data, y_test_data: Test data and labels.
    """
    y_pred = model.predict(X_val_data)

    print(f"Evaluation: {model.__class__.__name__}\n")  
    print("Accuracy:", accuracy_score(y_val_data, y_pred))
    print("Precision:", precision_score(y_val_data, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_val_data, y_pred))
    print("Classification Report:\n", classification_report(y_val_data, y_pred))


In [5]:
# Data Preprocessing
df_train['review'] = df_train['review'].apply(clean_text)

display(df_train.head(10))

/tmp/ipykernel_42/2559869307.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,id,sentiment,review
0,5814_8,1,stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty...
1,2381_9,1,classic war worlds timothy hines entertaining film obviously goes great effort lengths faithfully recreate h g wells classic book mr hines succeeds watched film appreciated fact standard predictab...
2,7759_3,0,film starts manager nicholas bell giving welcome investors robert carradine primal park secret project mutating primal animal using fossilized dna like jurassik park scientists resurrect one natur...
3,3630_4,0,must assumed praised film greatest filmed opera ever read somewhere either care opera care wagner care anything except desire appear cultured either representation wagner swan song movie strikes u...
4,9495_8,1,superbly trashy wondrously unpretentious 80 exploitation hooray pre credits opening sequences somewhat give false impression dealing serious harrowing drama need fear barely ten minutes later neck...
5,8196_8,1,know people think bad movie got pretty good plot good action change location harry hurt either sure offensive gratuitous movie like eastwood good form dirty harry liked pat hingle movie small town...
6,7166_2,0,movie could good comes way short cheesy special effects acting could looked past story lousy background story would better plot centers around evil druid witch linked woman gets migraines movie dr...
7,10633_1,0,watched video friend house glad waste money buying one video cover scene 1975 movie capricorn one movie starts several clips rocket blow ups related manned flight sibrel smoking gun short video cl...
8,319_1,0,friend mine bought film 1 even grossly overpriced despite featuring big names adam sandler billy bob thornton incredibly talented burt young film funny taking chisel hammering straight earhole use...
9,8713_10,1,movie full references like mad max ii wild one many others ladybug face clear reference tribute peter lorre movie masterpiece talk much future


In [6]:
# Feature Engineering: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5_000)

X = vectorizer.fit_transform(df_train['review'])
y = df_train['sentiment']

In [7]:
# Model Building: split data
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [8]:
# Model Building: Support Vector Machine
model_svm = SVC()
model_svm.fit(X_train, y_train)

# Evaluation
evaluate_trained_model(
    model_svm,
    X_val,
    y_val
)

Evaluation: SVC

Accuracy: 0.8884
Precision: 0.8804811796662786
Confusion Matrix:
 [[2173  308]
 [ 250 2269]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.88      0.89      2481
           1       0.88      0.90      0.89      2519

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000

